In [1]:
import pandas as pd

cities = pd.read_csv('cities.csv')
providers = pd.read_csv('providers.csv')
stations = pd.read_csv('stations.csv')
ticket_data = pd.read_csv('ticket_data.csv')

# Task 1:

- Extraire des infos intéressantes type prix min, moyen et max, durée min/max/moyenne par trajet

In [2]:
# look for data
print('rows: {}, columns: {}'.format(ticket_data.shape[0], ticket_data.shape[1]))
ticket_data.head(3)

rows: 74168, columns: 12


,id,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,o_city,d_city
0,6795025,8385,NaN,NaN,2017-10-13 14:00:00+00,2017-10-13 20:10:00+00,4550,2017-10-01 00:13:31.327+00,NaN,NaN,611,542
1,6795026,9,63.0,1044.0,2017-10-13 13:05:00+00,2017-10-14 06:55:00+00,1450,2017-10-01 00:13:35.773+00,"{149,418}",{13},611,542
2,6795027,8377,5905.0,6495.0,2017-10-13 13:27:00+00,2017-10-14 21:24:00+00,7400,2017-10-01 00:13:40.212+00,"{798,798,6794,6246}","{8377,8376}",611,542


In [3]:
# describe some statistic in numerical features
ticket_data.describe()

,id,company,o_station,d_station,price_in_cents,o_city,d_city
count,7.416800e+04,74168.000000,32727.000000,32727.000000,74168.000000,74168.000000,74168.000000
mean,6.832108e+06,7109.565527,2907.132673,2347.858129,4382.711061,849.186105,883.776265
std,2.141060e+04,3005.380792,3347.629180,3090.798007,3739.325367,1485.791782,1654.698575
min,6.795025e+06,9.000000,3.000000,3.000000,300.000000,5.000000,1.000000
25%,6.813567e+06,8376.000000,400.000000,396.000000,1900.000000,485.000000,453.000000
50%,6.832108e+06,8385.000000,701.000000,575.000000,3350.000000,628.000000,562.000000
75%,6.850650e+06,8385.000000,6246.000000,4538.000000,5250.000000,628.000000,628.000000
max,6.869192e+06,8387.000000,11017.000000,11017.000000,38550.000000,12190.000000,12190.000000


In [4]:
# info about types
ticket_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74168 entries, 0 to 74167
Data columns (total 12 columns):
id                 74168 non-null int64
company            74168 non-null int64
o_station          32727 non-null float64
d_station          32727 non-null float64
departure_ts       74168 non-null object
arrival_ts         74168 non-null object
price_in_cents     74168 non-null int64
search_ts          74168 non-null object
middle_stations    32727 non-null object
other_companies    32727 non-null object
o_city             74168 non-null int64
d_city             74168 non-null int64
dtypes: float64(2), int64(5), object(5)
memory usage: 6.8+ MB


In [5]:
#conver dates to datetime format
ticket_data['arrival_ts'] = pd.to_datetime(ticket_data['arrival_ts'])
ticket_data['departure_ts'] = pd.to_datetime(ticket_data['departure_ts'])

ticket_data.dtypes

id                               int64
company                          int64
o_station                      float64
d_station                      float64
departure_ts       datetime64[ns, UTC]
arrival_ts         datetime64[ns, UTC]
price_in_cents                   int64
search_ts                       object
middle_stations                 object
other_companies                 object
o_city                           int64
d_city                           int64
dtype: object

In [6]:
# create new column 'duration'. It is difference betweeen  arrival_ts - departure_ts in seconds. 
ticket_data = ticket_data.assign(duration = (ticket_data['arrival_ts'] - ticket_data['departure_ts']).dt.total_seconds())

# Extraire des infos intéressantes type prix min, moyen et max, durée min/max/moyenne par trajet
# !!! duartion in seconds
task1 = ticket_data.groupby(['o_city', 'd_city'])['price_in_cents', 'duration'].agg(
    {"price_in_cents":[min, max, 'mean'],"duration":[min, max, 'mean']})
task1 = task1.rename(columns = {'duration': 'duration_in_seconds'})
task1.head(20)

price_in_cents                      duration_in_seconds  \
                         min    max          mean                 min   
o_city d_city                                                           
5      23              18600  22000  20320.000000             31980.0   
6      227              9860  13650  11755.000000             44640.0   
       504              2000   8920   4042.666667             20160.0   
       628              2600   3190   2797.500000             34800.0   
       845               700   2420    864.626866              3600.0   
       10480            3700   4050   3833.333333             18000.0   
11     28               5090   5290   5190.000000             52500.0   
       33               2000   3280   2630.000000             78300.0   
       200              2390  10740   6405.238095             14760.0   
       424              1600   7100   4389.393939             28800.0   
       453              4890  16000   8990.000000             16260.0   
       485              1400   6500   2501.329787             10800.0   
       524              2790  17600   9558.000000             24600.0   
       580              1100   4600   2475.416667              6000.0   
       628              1600  14900   5724.961240             14580.0   
       639              1200   2500   2020.000000             13800.0   
       843               500   1890    627.704918              3000.0   
       1088             2650   3700   3239.090909             14400.0   
       2621             6150   9990   8318.333333             36600.0   
       5520             3100   3950   3487.500000             16800.0   

                                       
                    max          mean  
o_city d_city                          
5      23       57240.0  37128.000000  
6      227      54060.0  49350.000000  
       504      44400.0  29844.000000  
       628      52200.0  43800.000000  
       845      15060.0   4794.626866  
       10480    28800.0  21000.000000  
11     28       54600.0  53550.000000  
       33       96300.0  86940.000000  
       200     114000.0  41185.714286  
       424     153600.0  57454.545455  
       453     139800.0  44665.161290  
       485      74100.0  14368.085106  
       524      42600.0  31896.000000  
       580      39660.0  13897.500000  
       628     104700.0  37539.534884  
       639      70200.0  44775.000000  
       843      51600.0   4445.901639  
       1088     72900.0  23100.000000  
       2621     92400.0  68750.000000  
       5520     20400.0  17850.000000

# TASK 2
   - différence de prix moyen et durée selon le train, le bus et le covoit selon la distance du trajet (0-200km, 201-800km, 800-2000km, 2000+km) 

In [7]:
# drop unececary columns and rename some columns
ticket_data.drop(['departure_ts', 'arrival_ts', 'search_ts', 'middle_stations', 'other_companies', 'o_station', 'd_station'], 
                 axis=1, inplace=True)
ticket_data.rename(columns={'duration': 'duration_in_sec'}, inplace=True)

ticket_data.head()

,id,company,price_in_cents,o_city,d_city,duration_in_sec
0,6795025,8385,4550,611,542,22200.0
1,6795026,9,1450,611,542,64200.0
2,6795027,8377,7400,611,542,115020.0
3,6795028,8377,13500,611,542,77700.0
4,6795029,8381,7710,611,542,78360.0


In [8]:
# drop unececary columns and rename some columns
cities.drop(['population'], axis=1, inplace=True)
cities.head(1)

,id,local_name,unique_name,latitude,longitude
0,5159,"Padua, Veneto, Italia",padua,45.406435,11.876761


In [9]:
print('ticket: ', ticket_data.shape)
print('citites: ', cities.shape)

ticket:  (74168, 6)
citites:  (8040, 5)


In [10]:
# join ticket_data and citiyes by origin city
task2 = ticket_data.merge(cities, left_on = 'o_city', right_on = 'id', suffixes=('_left', '_right'))
# rename some columns
task2.rename(columns = {'unique_name': 'o_city_name', 'latitude': 'o_lat', 'longitude': 'o_lon'}, inplace=True)
task2 = task2.drop(['id_right', 'local_name'], axis=1)

task2.head()

,id_left,company,price_in_cents,o_city,d_city,duration_in_sec,o_city_name,o_lat,o_lon
0,6795025,8385,4550,611,542,22200.0,orleans,47.907018,1.90627
1,6795026,9,1450,611,542,64200.0,orleans,47.907018,1.90627
2,6795027,8377,7400,611,542,115020.0,orleans,47.907018,1.90627
3,6795028,8377,13500,611,542,77700.0,orleans,47.907018,1.90627
4,6795029,8381,7710,611,542,78360.0,orleans,47.907018,1.90627


In [11]:
# join ticket_data and citiyes by departure city
task2 = task2.merge(cities, left_on = 'd_city', right_on = 'id')

task2.rename(columns = {'unique_name': 'd_city_name', 'latitude': 'd_lat', 'longitude': 'd_lon'}, inplace=True)
task2 = task2.drop(['id', 'local_name'], axis=1)
task2.head()


,id_left,company,price_in_cents,o_city,d_city,duration_in_sec,o_city_name,o_lat,o_lon,d_city_name,d_lat,d_lon
0,6795025,8385,4550,611,542,22200.0,orleans,47.907018,1.90627,montpellier,43.604452,3.918318
1,6795026,9,1450,611,542,64200.0,orleans,47.907018,1.90627,montpellier,43.604452,3.918318
2,6795027,8377,7400,611,542,115020.0,orleans,47.907018,1.90627,montpellier,43.604452,3.918318
3,6795028,8377,13500,611,542,77700.0,orleans,47.907018,1.90627,montpellier,43.604452,3.918318
4,6795029,8381,7710,611,542,78360.0,orleans,47.907018,1.90627,montpellier,43.604452,3.918318


In [12]:
# drop some columns from providers data
providers = providers.loc[:,['id', 'name', 'transport_type']]
providers.head()

,id,name,transport_type
0,9,ouibus,bus
1,10,deinbus,bus
2,11,infobus,bus
3,12,studentAgency,bus
4,13,flixbus,bus


In [13]:
# join providers with our data by company id
task2 = task2.merge(providers, left_on='company', right_on='id')

task2 = task2.rename(columns = {'name': 'providers'})
task2.drop(['company', 'id'], axis=1, inplace=True)
task2.head()

,id_left,price_in_cents,o_city,d_city,duration_in_sec,o_city_name,o_lat,o_lon,d_city_name,d_lat,d_lon,providers,transport_type
0,6795025,4550,611,542,22200.0,orleans,47.907018,1.906270,montpellier,43.604452,3.918318,bbc,carpooling
1,6797839,5800,628,542,28800.0,paris,48.847702,2.352311,montpellier,43.604452,3.918318,bbc,carpooling
2,6797840,5450,628,542,28800.0,paris,48.847702,2.352311,montpellier,43.604452,3.918318,bbc,carpooling
3,6797841,5700,628,542,27600.0,paris,48.847702,2.352311,montpellier,43.604452,3.918318,bbc,carpooling
4,6797842,4750,628,542,31800.0,paris,48.847702,2.352311,montpellier,43.604452,3.918318,bbc,carpooling


In [16]:
# create function for calculating distance. haversine_distance
import numpy as np
def haversine_distance(lat1, lon1, lat2, lon2):
    r = 6371 #earth radius
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
    res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
    return np.round(res, 2)

In [17]:
# create new column 'distance' and calcuate distance by orogin(lat, lon) and destination(lat, lon)
task2 = task2.assign(distance = haversine_distance(task2.o_lat, task2.o_lon, task2.d_lat, task2.d_lon))
task2.head()

,id_left,price_in_cents,o_city,d_city,duration_in_sec,o_city_name,o_lat,o_lon,d_city_name,d_lat,d_lon,providers,transport_type,distance
0,6795025,4550,611,542,22200.0,orleans,47.907018,1.906270,montpellier,43.604452,3.918318,bbc,carpooling,503.2
1,6797839,5800,628,542,28800.0,paris,48.847702,2.352311,montpellier,43.604452,3.918318,bbc,carpooling,595.3
2,6797840,5450,628,542,28800.0,paris,48.847702,2.352311,montpellier,43.604452,3.918318,bbc,carpooling,595.3
3,6797841,5700,628,542,27600.0,paris,48.847702,2.352311,montpellier,43.604452,3.918318,bbc,carpooling,595.3
4,6797842,4750,628,542,31800.0,paris,48.847702,2.352311,montpellier,43.604452,3.918318,bbc,carpooling,595.3


In [18]:
# function mapping categories in distance
def cat_by_dist(data):
    if data <= float(200):
        data = '0-200'
    elif data > float(200) and data <=float(800):
        data = '200-800'
    else:
        data = '800-2000'
    return data

task2['distance_cat'] = task2['distance'].apply(cat_by_dist)


task2  = task2.groupby(['transport_type', 'distance_cat']).price_in_cents.agg(['mean', max, min])
# answer is
task2

mean    max   min
transport_type distance_cat                           
bus            0-200          2182.830918  22900   850
               200-800        3494.850022  22480  1000
               800-2000       6949.950045  17400  2290
carpooling     0-200          1177.352385  12850   300
               200-800        3218.064310  13800   850
               800-2000       8626.545455  16150  4496
train          0-200          3487.900779  25100   490
               200-800        9124.779489  38550  1400
               800-2000      15463.746130  37550  1940